In [7]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [2]:
def get_metadata_features(i):
    """
    Obtain the dataset
    """
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_train_' + str(i) + '.pkl')
    #print(df_X_train.head())
    df_X_test= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    X_train_meta = df_X_train.drop(['Previous_User_Utterance'], axis=1)
    X_test_meta = df_X_test.drop(['Previous_User_Utterance'], axis=1)
    
    #print(X_train_meta['Previous_Speech_Act'].unique())
    #print(X_train_meta['Previous_Search_Act'].unique())
    
    return X_train_meta, X_test_meta, df_Y_train, df_Y_test

In [3]:
from keras.engine.topology import Layer
class AttentionLayer(Layer):
    
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weight = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

In [4]:
def create_bilstm_model_meta(reshaped_data):
    samples,timesteps,features = reshaped_data.shape
    input_data = Input(shape=(timesteps,features))
    # lstm needs (samples,timesteps,features) tensor as the input
    x = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(input_data) 
    #x = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25))(input_data) 
    x, sent_coeffs = AttentionLayer(features,return_coefficients=True,name='sent_attention')(x)
    #x = GlobalMaxPool1D()(x)
    #x = Dense(100, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(4, activation="softmax")(x) #output layer
    model = Model(inputs=input_data, outputs=x)
    return model  

In [14]:
def execute_bilstm_metadata(i):
    df_train_meta, df_test_meta, df_Y_train, df_Y_test = get_metadata_features(i)
    train_meta = df_train_meta.values
    test_meta = df_test_meta.values
    print("Shape check post file read:")
    #print(train_meta)
    print(train_meta.shape, test_meta.shape)   
    
    timestamp = 1 #number of successive sequences combined
    
    r, c = df_train_meta.shape
    staggering = r - timestamp + 1 # final number of instances generated 
    X_train_reshaped = np.concatenate([train_meta[x:x+timestamp,:] for x in range(r-timestamp+1)])
    X_train_reshaped = X_train_reshaped.reshape(staggering, timestamp, c) # c is the number of features
    #print(X_train_reshaped)
    print("X_train_reshaped.shape:")
    print(X_train_reshaped.shape)
    
    r2, c2 = df_test_meta.shape
    staggering2 = r2 - timestamp + 1 # final number of instances generated 
    X_test_reshaped = np.concatenate([test_meta[x:x+timestamp,:] for x in range(r2-timestamp+1)])
    X_test_reshaped = X_test_reshaped.reshape(staggering2, timestamp, c2) # c is the number of features
    #print(X_test_reshaped)
    print("X_test_reshaped.shape:")
    print(X_test_reshaped.shape)
    
    df_Y_train = df_Y_train.iloc[timestamp-1:,]
    df_Y_test = df_Y_test.iloc[timestamp-1:,]
    
    print("Shape check for train and test sets post reshaping for lstm:")
    print(X_train_reshaped.shape, X_test_reshaped.shape)
    print(df_Y_train.shape, df_Y_test.shape)
    
    model = create_bilstm_model_meta(X_train_reshaped)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    model.summary()
    
    """
    encoding the output labels
    """
    encoder = LabelEncoder()
    encoder.fit(df_Y_train)
    training_op_labels_encoded = encoder.transform(df_Y_train)
    test_op_labels_encoded = encoder.transform(df_Y_test)
    print("Training and test output labels(before one-hot/after enc)")
    print(training_op_labels_encoded.shape, test_op_labels_encoded.shape)
    """
    converting the output labels to one-hot form
    """
    training_op_labels_onehot= np_utils.to_categorical(training_op_labels_encoded)
    test_op_labels_onehot = np_utils.to_categorical(test_op_labels_encoded)

    #print(training_op_labels_onehot)
    print("Training and test output labels(after one-hot)")
    print(training_op_labels_onehot.shape, len(training_op_labels_onehot))
    print(test_op_labels_onehot.shape, len(test_op_labels_onehot))

    print("X_train_reshaped", X_train_reshaped.shape)
    print("training_op_labels_onehot", training_op_labels_onehot.shape)

    X_test_reshaped = np.asarray(X_test_reshaped, dtype='float32')
    X_train_reshaped = np.asarray(X_train_reshaped, dtype='float32')
    model.fit(X_train_reshaped, training_op_labels_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

    """
    evaluate model
    """
    print(model.metrics_names)
    loss, binary_accuracy, categorical_accuracy = model.evaluate(X_test_reshaped, y = test_op_labels_onehot, batch_size=BATCH_SIZE, verbose=1)
    
    print(loss, binary_accuracy, categorical_accuracy)

    """
    predict the probabilty of output classes
    and pick the best one
    """
    
    prediction_vector = model.predict(X_test_reshaped, batch_size=BATCH_SIZE, verbose=1)
    predicted_classes = np.argmax(prediction_vector, axis=1)
    original_classes = np.argmax(test_op_labels_onehot, axis=1)
    
    """
    # verification of correctness:
    total_correct = sum(original_classes == predicted_classes)
    print("Total number of correct predictions:",total_correct)
    print("Accuracy:",total_correct/len(test_op_labels_onehot))
    acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
    print('Overall accuracy: {} %'.format(acc*100))
    """
    conf_mat = confusion_matrix(predicted_classes, original_classes)
    print(conf_mat)

    """
    # show the inputs and predicted outputs
    list_bilstm_prediction = []    
    for i in range(len(X_test_reshaped)):
        row = []
        row.extend(X_test_reshaped[i])
        row.extend(test_op_labels_onehot[i])
        list_bilstm_prediction.append(row)
    df_bilstm_prediction = pd.DataFrame(data=list_bilstm_prediction)
    df_bilstm_prediction.to_pickle("nlp_bilstm_predictions.pkl")
    """
       
    return predicted_classes, categorical_accuracy, binary_accuracy, conf_mat  

In [17]:
"""
import tensorflow as tf
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True
"""

import pandas as pd
import numpy as np 
from numpy import array

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.layers.merge import concatenate
from keras.models import Model, model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils, to_categorical
from keras.engine.topology import Layer
from keras import backend as K

# checkpoint
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Attention

BATCH_SIZE=32
EPOCHS=1 #300

file = open('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-attn/bilstm_meta_attn_op.txt','w') #overwrites previous
file.close()

if __name__== "__main__":
    df_prediction = pd.DataFrame()
    df_accuracy =  pd.DataFrame()
    file = open('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-attn/bilstm_meta_attn_op.txt','a') #append mode 
    for i in range(1,3):
        outputname = 'meta_attn_'+ str(i)        
        predictions, acc_cat, acc_bin, conf_matrix = execute_bilstm_metadata(i)
        df_prediction[outputname] = predictions
        df_accuracy[i] = [acc_cat]
        file.write("\nIteration:" + str(i) + "\nCategorical Accuracy:" + str(acc_cat) + 
                   "\nBinary Accuracy:" + str(acc_bin) + "\nConfusion Matrix:\n" + str(conf_matrix) + "\n\n")
        df_prediction.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-attn/predictions_bilstm_attn_meta_' + str(i) + '.csv')    
        df_accuracy.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-attn/accuracy_bilstm_attn_meta_' + str(i) + '.csv') 
        
    df_prediction.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-attn/predictions_bilstm_attn_meta.csv')    
    df_accuracy.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-attn/accuracy_bilstm_attn_meta.csv')    
    file.close()

Shape check post file read:
(406, 8) (102, 8)
X_train_reshaped.shape:
(406, 1, 8)
X_test_reshaped.shape:
(102, 1, 8)
Shape check for train and test sets post reshaping for lstm:
(406, 1, 8) (102, 1, 8)
(406, 1) (102, 1)
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1, 8)]            0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 1, 64)             10496     
_________________________________________________________________
sent_attention (AttentionLay [(None, 64), (None, 1, 1) 528       
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 260       
Total params: 11,284
Trainable 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


12/12 [==============================] - 1s 47ms/step - loss: 1.4499 - binary_accuracy: 0.7500 - categorical_accuracy: 0.1863 - val_loss: 1.3435 - val_binary_accuracy: 0.7500 - val_categorical_accuracy: 0.2195
['loss', 'binary_accuracy', 'categorical_accuracy']
4/4 [==============================] - 0s 2ms/step - loss: 1.4095 - binary_accuracy: 0.7500 - categorical_accuracy: 0.2353
1.4094624519348145 0.75 0.23529411852359772
4/4 [==============================] - 0s 2ms/step
[[ 5  3  5  0]
 [ 2  4  3  2]
 [19 14 12  5]
 [11  8  6  3]]
Shape check post file read:
(406, 8) (102, 8)
X_train_reshaped.shape:
(406, 1, 8)
X_test_reshaped.shape:
(102, 1, 8)
Shape check for train and test sets post reshaping for lstm:
(406, 1, 8) (102, 1, 8)
(406, 1) (102, 1)
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1, 8)]            0         
__________________

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


12/12 [==============================] - 1s 48ms/step - loss: 1.3807 - binary_accuracy: 0.7452 - categorical_accuracy: 0.2493 - val_loss: 1.4242 - val_binary_accuracy: 0.7500 - val_categorical_accuracy: 0.2683
['loss', 'binary_accuracy', 'categorical_accuracy']
4/4 [==============================] - 0s 2ms/step - loss: 1.3626 - binary_accuracy: 0.7500 - categorical_accuracy: 0.2941
1.3625977039337158 0.75 0.29411765933036804
4/4 [==============================] - 0s 2ms/step
[[28 19 36  6]
 [ 4  0  6  1]
 [ 0  0  2  0]
 [ 0  0  0  0]]
